<h1> IBM Data Science Capstone Project</h1>
<h5> Author : Voranipit </h5>
This notebook  will mainly use for IBM course assignment

<h3> Introduction and Bussiness Problem </h3>

   <b>Problem </b> : <br>           Which bussiness will be recommended to run in Toronto neighborhood ,why it will profitable?<br>
,What is target customer group ? and how to do it efficienctly? <br>

<b>Which data do you need to solve the problem ?<br></b>
We need data set that contain category of venues in Toronto as much as possible.<br>

<b>Where is the data sources ?<br></b>
- Post code , neighborhood of each borough from wikipedia in order to specified which neighborhood is in Toronto Neighborhood.<br>
- FoursquareAPI in order to get venues in each neighborhood.<br>



<b>Which format of data set we need ?<br></b>
Rows of each neighborhood in Toronto and the columns of Top 10 most common venues in each neighborhood(rows)<br>

<b>How data will use for solve the problem ?<br></b>
Once we know cluster of other venues we will find inside of the data that why in that cluster has that most common venues .<br>
when we know the reason we can use that reason to find out which bussiness will be profitable.<br>

<h3> Data  Source</h3>

In [1]:
import pandas as pd
import numpy as np
from jupyterthemes import jtplot
jtplot.style(theme = 'monokai',ticks = True,grid = False ,context = 'notebook')

<h3> Model Clustering (K-mean) </h3>

<h3> Identify clusters </h3>

<h3> Seeking For oppotunity </h3>

<h3>How to use these oppotunity ? </h3>

<h3> Conclusion and actionable recommendation.<h3> 